<a href="https://colab.research.google.com/github/Rohith2830/NYC-Taxi-Dataset-Analysis/blob/main/NYC_Taxi_Dataset_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt-get install openjdk-11-jdk-headless -qq

In [6]:
!wget https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz


--2025-07-11 12:12:17--  https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-11 12:12:17 ERROR 404: Not Found.



In [8]:
# prompt: !wget https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install findspark

tar: spark-3.3.2-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [10]:
!pip install -q findspark


In [12]:
# Download a valid Spark version (example using Spark 3.3.0)
!wget https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# Extract Spark
!tar -xvzf spark-3.3.0-bin-hadoop3.tgz > /dev/null

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

# Install and initialize findspark
!pip install -q findspark
import findspark
findspark.init()

--2025-07-11 12:13:57--  https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299321244 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop3.tgz’

spark-3.3.0-bin-had 100%[===================>] 285.45M  24.5MB/s    in 11s     

2025-07-11 12:14:09 (25.0 MB/s) - ‘spark-3.3.0-bin-hadoop3.tgz’ saved [299321244/299321244]



In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYC_Taxi_Analysis").getOrCreate()


In [14]:
df = spark.read.csv("/content/yellow_tripdata_2020-01.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+---

In [15]:
from pyspark.sql.functions import col

df_with_revenue = df.withColumn("Revenue",
    col("fare_amount") + col("extra") + col("mta_tax") +
    col("improvement_surcharge") + col("tip_amount") +
    col("tolls_amount") + col("total_amount")
)

df_with_revenue.select("fare_amount", "tip_amount", "total_amount", "Revenue").show(5)


+-----------+----------+------------+-------+
|fare_amount|tip_amount|total_amount|Revenue|
+-----------+----------+------------+-------+
|        6.0|      1.47|       11.27|  22.54|
|        7.0|       1.5|        12.3|   24.6|
|        6.0|       1.0|        10.8|   21.6|
|        5.5|      1.36|        8.16|  16.32|
|        3.5|       0.0|         4.8|    9.6|
+-----------+----------+------------+-------+
only showing top 5 rows



In [16]:
df_with_revenue.groupBy("PULocationID") \
    .sum("passenger_count") \
    .orderBy("sum(passenger_count)", ascending=False) \
    .show(10)


+------------+--------------------+
|PULocationID|sum(passenger_count)|
+------------+--------------------+
|         237|              433243|
|         161|              425986|
|         236|              403347|
|         230|              360096|
|         162|              351011|
|         186|              338952|
|         132|              326402|
|          48|              297148|
|         142|              294502|
|         170|              289593|
+------------+--------------------+
only showing top 10 rows



In [17]:
df_with_revenue.groupBy("VendorID") \
    .avg("total_amount") \
    .withColumnRenamed("avg(total_amount)", "avg_total_earning") \
    .orderBy("VendorID") \
    .show()


+--------+------------------+
|VendorID| avg_total_earning|
+--------+------------------+
|    null|37.217091425863046|
|       1|18.113429405148675|
|       2|18.648347164220418|
+--------+------------------+



In [18]:
from pyspark.sql.functions import col, to_timestamp

df_time = df_with_revenue.withColumn("pickup_time", to_timestamp("tpep_pickup_datetime"))


In [19]:
from pyspark.sql.functions import window

df_time.groupBy(
    window("pickup_time", "1 hour"), "payment_type"
).count().orderBy("window").show(10, truncate=False)


+------------------------------------------+------------+-----+
|window                                    |payment_type|count|
+------------------------------------------+------------+-----+
|{2003-01-01 00:00:00, 2003-01-01 01:00:00}|2           |1    |
|{2008-12-31 23:00:00, 2009-01-01 00:00:00}|1           |5    |
|{2008-12-31 23:00:00, 2009-01-01 00:00:00}|2           |5    |
|{2009-01-01 00:00:00, 2009-01-01 01:00:00}|2           |12   |
|{2009-01-01 00:00:00, 2009-01-01 01:00:00}|1           |1    |
|{2009-01-01 01:00:00, 2009-01-01 02:00:00}|2           |2    |
|{2009-01-01 02:00:00, 2009-01-01 03:00:00}|2           |2    |
|{2009-01-01 04:00:00, 2009-01-01 05:00:00}|2           |1    |
|{2009-01-01 10:00:00, 2009-01-01 11:00:00}|2           |1    |
|{2019-12-18 15:00:00, 2019-12-18 16:00:00}|1           |2    |
+------------------------------------------+------------+-----+
only showing top 10 rows



In [20]:
from pyspark.sql.functions import to_date

df_date = df_with_revenue.withColumn("trip_date", to_date("tpep_pickup_datetime"))


In [21]:
from pyspark.sql.functions import sum as _sum

top_vendors = df_date.groupBy("VendorID", "trip_date") \
    .agg(
        _sum("total_amount").alias("total_earning"),
        _sum("passenger_count").alias("total_passengers"),
        _sum("trip_distance").alias("total_distance")
    ) \
    .orderBy("trip_date", _sum("total_amount").desc())

top_vendors.show(5, truncate=False)


+--------+----------+------------------+----------------+------------------+
|VendorID|trip_date |total_earning     |total_passengers|total_distance    |
+--------+----------+------------------+----------------+------------------+
|2       |2003-01-01|0.0               |1               |0.0               |
|2       |2008-12-31|268.75            |17              |62.83999999999999 |
|2       |2009-01-01|441.20000000000005|30              |93.00999999999999 |
|2       |2019-12-18|9.11              |5               |0.0               |
|2       |2019-12-31|2323.0799999999986|287             |414.07000000000016|
+--------+----------+------------------+----------------+------------------+
only showing top 5 rows



In [22]:
from pyspark.sql.functions import sum as _sum

df_with_revenue.groupBy("PULocationID", "DOLocationID") \
    .agg(_sum("passenger_count").alias("total_passengers")) \
    .orderBy("total_passengers", ascending=False) \
    .show(10)


+------------+------------+----------------+
|PULocationID|DOLocationID|total_passengers|
+------------+------------+----------------+
|         237|         236|           67885|
|         236|         236|           57662|
|         236|         237|           56488|
|         237|         237|           49757|
|         264|         264|           44789|
|         239|         238|           30402|
|         239|         142|           28755|
|         161|         237|           27492|
|         142|         239|           27260|
|         186|         230|           25857|
+------------+------------+----------------+
only showing top 10 rows



In [23]:
from pyspark.sql.functions import max

latest_time = df_with_revenue.select(max("tpep_pickup_datetime")).collect()[0][0]
print("Latest pickup time:", latest_time)


Latest pickup time: 2021-01-02 01:12:10


In [25]:
from pyspark.sql.functions import to_timestamp, col
from datetime import timedelta

df_recent = df_with_revenue.withColumn("pickup_time", to_timestamp("tpep_pickup_datetime"))

# Calculate the timestamp 10 seconds before the latest time
time_threshold = latest_time - timedelta(seconds=10)

df_last10s = df_recent.filter(
    col("pickup_time") >= time_threshold
)

df_last10s.groupBy("PULocationID") \
    .agg({"passenger_count": "sum"}) \
    .orderBy("sum(passenger_count)", ascending=False) \
    .show()

+------------+--------------------+
|PULocationID|sum(passenger_count)|
+------------+--------------------+
|          90|                   1|
+------------+--------------------+

